# ECT 2026 Thailand Election — Google Drive File Extractor

Recursively list all files in the ECT 2026 election results Google Drive folder and export metadata as JSONL.

**Target folder**: `https://drive.google.com/drive/folders/1elYwd_ATWpm8q_ZxoxKcXO1TaVF7MElj`

**Output fields per file**:
| Field | Description |
|---|---|
| `province_name` | Thai province inferred from folder path |
| `path` | Full path from root folder |
| `size_mb` | File size in MB |
| `file_id` | Google Drive file ID |
| `folder_id` | Parent folder ID |

---

### Prerequisites

Run the following in a terminal **before** opening this notebook:

```bash
gcloud auth application-default login \
  --scopes="openid,https://www.googleapis.com/auth/userinfo.email,https://www.googleapis.com/auth/cloud-platform,https://www.googleapis.com/auth/drive.readonly"
```

Then enable the Drive API in your project:
```bash
gcloud services enable drive.googleapis.com
```

## 1. Install Dependencies

In [1]:
!pip install -q google-api-python-client google-auth google-auth-oauthlib google-auth-httplib2 pandas

## 2. Authenticate & Build Drive Client

In [2]:
import os
import json
import time
from pathlib import Path
from typing import Optional

import google.auth
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# ── Target folder ──────────────────────────────────────────────────────────────
ROOT_FOLDER_ID = "1elYwd_ATWpm8q_ZxoxKcXO1TaVF7MElj"

# ── Output ─────────────────────────────────────────────────────────────────────
OUTPUT_FILE = "datasets/ect_2026_drive_files.jsonl"

print(f"Root Folder ID : {ROOT_FOLDER_ID}")
print(f"Output File    : {OUTPUT_FILE}")

Root Folder ID : 1elYwd_ATWpm8q_ZxoxKcXO1TaVF7MElj
Output File    : datasets/ect_2026_drive_files.jsonl


In [6]:
SCOPES = ["https://www.googleapis.com/auth/drive.readonly"]

# ── Option A: Application Default Credentials (ADC) ───────────────────────────
# Requires: gcloud auth application-default login --scopes="...drive.readonly"
try:
    credentials, project = google.auth.default(scopes=SCOPES)
    drive_service = build("drive", "v3", credentials=credentials)
    # Quick connectivity check
    drive_service.about().get(fields="user").execute()
    print(f"✅ Authenticated via ADC (project: {project})")
except Exception as e:
    print(f"⚠️  ADC failed: {e}")
    print()
    print("Run this in your terminal, then restart the kernel:")
    print()
    print('  gcloud auth application-default login \\')
    print('    --scopes="openid,https://www.googleapis.com/auth/userinfo.email,')
    print('              https://www.googleapis.com/auth/cloud-platform,')
    print('              https://www.googleapis.com/auth/drive.readonly"')
    raise

✅ Authenticated via ADC (project: datadog-ese-sandbox)


## 3. Recursive Traversal

In [7]:
FOLDER_MIME = "application/vnd.google-apps.folder"


def list_folder_contents(service, folder_id: str) -> list[dict]:
    """Paginated listing of a Drive folder's direct children."""
    items, page_token = [], None
    while True:
        resp = (
            service.files()
            .list(
                q=f"'{folder_id}' in parents and trashed = false",
                fields="nextPageToken, files(id, name, mimeType, size, parents)",
                pageToken=page_token,
                pageSize=1000,
                supportsAllDrives=True,
                includeItemsFromAllDrives=True,
            )
            .execute()
        )
        items.extend(resp.get("files", []))
        page_token = resp.get("nextPageToken")
        if not page_token:
            break
    return items


def infer_province(path: str) -> Optional[str]:
    """
    Infer province from the folder path.
    ECT structure: <Root> / <Province> / ... / file
    Province is the second path component (index 1).
    """
    parts = [p.strip() for p in path.split("/") if p.strip()]
    return parts[1] if len(parts) >= 2 else None


def traverse(
    service,
    folder_id: str,
    current_path: str = "",
    results: Optional[list] = None,
    depth: int = 0,
) -> list[dict]:
    """Recursively collect file metadata from a Drive folder tree."""
    if results is None:
        results = []

    indent = "  " * depth
    for item in list_folder_contents(service, folder_id):
        name = item["name"]
        item_id = item["id"]
        path = f"{current_path}/{name}" if current_path else name

        if item["mimeType"] == FOLDER_MIME:
            print(f"{indent}📁 {name}/")
            traverse(service, item_id, path, results, depth + 1)
        else:
            size_bytes = int(item.get("size", 0))
            size_mb = round(size_bytes / (1024 * 1024), 4) if size_bytes else None
            results.append(
                {
                    "province_name": infer_province(path),
                    "path": path,
                    "size_mb": size_mb,
                    "file_id": item_id,
                    "folder_id": folder_id,
                }
            )
            print(f"{indent}📄 {name}  ({size_mb} MB)")

    return results


print("✅ Traversal functions ready")

✅ Traversal functions ready


## 4. Run Extraction

In [8]:
# Resolve root folder name
root_meta = (
    drive_service.files()
    .get(fileId=ROOT_FOLDER_ID, fields="id,name", supportsAllDrives=True)
    .execute()
)
root_name = root_meta["name"]
print(f"Root: {root_name}  (id: {ROOT_FOLDER_ID})")
print("\n🔍 Traversing...\n")

t0 = time.time()
all_files = traverse(drive_service, ROOT_FOLDER_ID, current_path=root_name)
elapsed = time.time() - t0

print(f"\n{'─'*60}")
print(f"✅  {len(all_files)} files found in {elapsed:.1f}s")

Root: ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)  (id: 1elYwd_ATWpm8q_ZxoxKcXO1TaVF7MElj)

🔍 Traversing...

📁 หนองคาย/
  📁 แบบแบ่งเขต/
    📄 70. หนองคาย เขต 3.pdf  (0.1196 MB)
    📄 70. หนองคาย เขต 2.pdf  (0.124 MB)
    📄 70. หนองคาย เขต 1.pdf  (0.1309 MB)
  📁 แบบบัญชีรายชื่อ/
    📄 70. หนองคาย เขต 3 (บช).pdf  (0.2377 MB)
    📄 70. หนองคาย เขต 2 (บช).pdf  (0.2521 MB)
    📄 70. หนองคาย เขต 1 (บช).pdf  (0.2663 MB)
📁 สุรินทร์/
  📁 แบบแบ่งเขต/
    📄 69. สุรินทร์ เขต8.pdf  (0.148 MB)
    📄 69. สุรินทร์ เขต3.pdf  (0.1757 MB)
    📄 69. สุรินทร์ เขต 2.pdf  (0.1242 MB)
    📄 69. สุรินทร์ เขต 4.pdf  (0.1369 MB)
    📄 69. สุรินทร์ เขต 7.pdf  (0.1133 MB)
    📄 69. สุรินทร์ เขต 1.pdf  (0.1126 MB)
    📄 69. สุรินทร์ เขต 6.pdf  (0.1144 MB)
    📄 69. สุรินทร์ เขต 5.pdf  (0.1317 MB)
  📁 แบบบัญชีรายชื่อ/
    📄 69. สุรินทร์ เขต8 บช.pdf  (0.1755 MB)
    📄 69. สุรินทร์ เขต3 บช.pdf  (0.2866 MB)
    📄 69. สุรินทร์ เขต 5 (บช).pdf  (0.2413 MB)
    📄 69. สุรินทร์ เขต 4 (บช).pdf  (0.2503 MB)
    📄 69. สุรินทร์ เขต 2 

## 5. Preview

In [9]:
import pandas as pd

df = pd.DataFrame(all_files)
print(f"Total files    : {len(df)}")
print(f"Unique provinces: {df['province_name'].nunique()}")
print()
df.head(10)

Total files    : 776
Unique provinces: 76



,province_name,path,size_mb,file_id,folder_id
0,หนองคาย,ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/หนองคาย/...,0.1196,1JDOkP6nW0qNfSg27DBX_7ZzZ4epF6u32,1EvhR0-EoN_vCkqo3R9xFsNp6ipA7YsHy
1,หนองคาย,ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/หนองคาย/...,0.1240,1bKuaF46ErxZZrQsbrFl-ri35t3SI47Do,1EvhR0-EoN_vCkqo3R9xFsNp6ipA7YsHy
2,หนองคาย,ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/หนองคาย/...,0.1309,1wHWZJl0oCsp8mAR2oF_O1xDYY--m8WAt,1EvhR0-EoN_vCkqo3R9xFsNp6ipA7YsHy
3,หนองคาย,ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/หนองคาย/...,0.2377,1zQQJm3O-vPlpj1xpk6gn-07a83XBTxzz,1ITySwBS-RMSOsVahucnByxG7clsOjHd0
4,หนองคาย,ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/หนองคาย/...,0.2521,15ZCUDRMUbILF3Zv1Z4gvw28Hw4sRRsNm,1ITySwBS-RMSOsVahucnByxG7clsOjHd0
5,หนองคาย,ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/หนองคาย/...,0.2663,1dCqfZ6ix_uBso92V_6HWJ3jJEQX53oI-,1ITySwBS-RMSOsVahucnByxG7clsOjHd0
6,สุรินทร์,ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/สุรินทร์...,0.1480,1-L64b43MsUtP0s0wQTwrbg7b5zCHHe2e,1ZXlZfWYya48fjl-VXngIsbjJYwzFVmJt
7,สุรินทร์,ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/สุรินทร์...,0.1757,115_l6se7XcVnV_wLjRCqxWpiUV3cgRUt,1ZXlZfWYya48fjl-VXngIsbjJYwzFVmJt
8,สุรินทร์,ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/สุรินทร์...,0.1242,16rtG2IsqSAOKHBFWhLvUnjTrtbpRwmJF,1ZXlZfWYya48fjl-VXngIsbjJYwzFVmJt
9,สุรินทร์,ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/สุรินทร์...,0.1369,1nB73PxQ4Nceg-M-MOA3P-Rc4C0iEWGtK,1ZXlZfWYya48fjl-VXngIsbjJYwzFVmJt


In [10]:
# Province-level summary
(
    df.groupby("province_name", dropna=False)
    .agg(file_count=("file_id", "count"), total_size_mb=("size_mb", "sum"))
    .assign(total_size_mb=lambda x: x["total_size_mb"].round(2))
    .sort_values("file_count", ascending=False)
    .reset_index()
)

,province_name,file_count,total_size_mb
0,กรุงเทพมหานคร,65,15.50
1,นครราชสีมา,32,5.20
2,ขอนแก่น,22,3.68
3,อุบลราชธานี,22,3.70
4,ชลบุรี,20,3.24
...,...,...,...
71,แม่ฮ่องสอน,4,0.67
72,สิงห์บุรี,2,1.11
73,ตราด,2,0.38
74,ระนอง,2,0.35


## 6. Export JSONL

In [12]:
out = Path(OUTPUT_FILE)
out.parent.mkdir(parents=True, exist_ok=True)

with out.open("w", encoding="utf-8") as f:
    for record in all_files:
        f.write(json.dumps(record, ensure_ascii=False) + "\n")

print(f"✅ {len(all_files)} records → {out}  ({out.stat().st_size / 1024:.1f} KB)")
print()
print("First 3 records:")
with out.open(encoding="utf-8") as f:
    for i, line in enumerate(f):
        if i >= 3:
            break
        print(line.rstrip())

✅ 776 records → datasets/ect_2026_drive_files.jsonl  (283.1 KB)

First 3 records:
{"province_name": "หนองคาย", "path": "ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/หนองคาย/แบบแบ่งเขต/70. หนองคาย เขต 3.pdf", "size_mb": 0.1196, "file_id": "1JDOkP6nW0qNfSg27DBX_7ZzZ4epF6u32", "folder_id": "1EvhR0-EoN_vCkqo3R9xFsNp6ipA7YsHy"}
{"province_name": "หนองคาย", "path": "ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/หนองคาย/แบบแบ่งเขต/70. หนองคาย เขต 2.pdf", "size_mb": 0.124, "file_id": "1bKuaF46ErxZZrQsbrFl-ri35t3SI47Do", "folder_id": "1EvhR0-EoN_vCkqo3R9xFsNp6ipA7YsHy"}
{"province_name": "หนองคาย", "path": "ประกาศผลการนับคะแนน สส. (8 ก.พ. 2569)/หนองคาย/แบบแบ่งเขต/70. หนองคาย เขต 1.pdf", "size_mb": 0.1309, "file_id": "1wHWZJl0oCsp8mAR2oF_O1xDYY--m8WAt", "folder_id": "1EvhR0-EoN_vCkqo3R9xFsNp6ipA7YsHy"}
